<a href="https://colab.research.google.com/github/Kimtaewookdev/Sec2_project/blob/main/AI_13_%EA%B9%80%ED%83%9C%EC%9A%B1_section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI_13 Section2 Project 
# League of Legends 승패 예측

### 1. 데이터 선정 이유 및 문제 정의



In [ ]:
!pip install category_encoders==2.*
!pip install eli5
!pip install --upgrade pandas-profiling
!pip install pdpbox
!pip install shap
!pip install --upgrade xgboost
!pip install --upgrade jinja2

In [151]:
import xgboost
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_profiling import profile_report

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
from scipy.stats import uniform, randint



warnings.filterwarnings("ignore")
xgboost.set_config(verbosity=0)

### 2. EDA

In [134]:
df = pd.read_csv('/content/drive/MyDrive/sec2 ds/lol_na_s11_highelo.csv')

# 불필요 칼럼 제거

dc1 = df.columns[['veteran' in x for x in df.columns]].tolist()
dc2 = df.columns[['inactive' in x for x in df.columns]].tolist()
dc3 = df.columns[['freshblood' in x for x in df.columns]].tolist()
dc4 = df.columns[['hotstreak' in x for x in df.columns]].tolist()
dc5 = df.columns[['wins' in x for x in df.columns]].tolist()
dc6 = df.columns[['losses' in x for x in df.columns]].tolist()
dc7 = df.columns[['avg_game_time' in x for x in df.columns]].tolist()

dc = dc1 + dc2 + dc3 + dc4 + dc5 + dc6 + dc7

wr = df.copy()

wr.drop(dc, axis = 1, inplace = True)

In [135]:
# 중복 열 확인
wr.duplicated(['gameid']).value_counts()

False    2088
dtype: int64

In [136]:
wr.drop('gameid', axis = 1, inplace = True)

In [137]:
# 티어, 랭크, lp = lp 포인트로 정리
# 결측치는 Simpleimputer mean으로 처리

def trans_rank(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_rank'
      wr[col][(wr[col] == 'I')] = -100
      wr[col][(wr[col] == 'II')] = -200
      wr[col][(wr[col] == 'III')] = -300
      wr[col][(wr[col] == 'IV')] = -400
#      wr[col].fillna(0, inplace = True)

def trans_tier(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_tier'
      wr[col][(wr[col] == 'CHALLENGER')] = 100
      wr[col][(wr[col] == 'GRANDMASTER')] = 100
      wr[col][(wr[col] == 'MASTER')] = 100
      wr[col][(wr[col] == 'DIAMOND')] = 0
      wr[col][(wr[col] == 'PLATINUM')] = -400
      wr[col][(wr[col] == 'GOLD')] = -800
      wr[col][(wr[col] == 'SILVER')] = -1200
#      wr[col].fillna(0, inplace = True)

def trans_lp(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      tier = f'{t}_' + p + '_tier'
      rank = f'{t}_' + p + '_rank'
      lp = f'{t}_' + p + '_lp'
#      wr[lp].fillna(0, inplace = True)
      wr[lp] = wr[tier] + wr[rank] + wr[lp]
      wr.drop([tier, rank], axis = 1, inplace = True)

trans_rank(wr)
trans_tier(wr)
trans_lp(wr)

wr.shape

(2088, 141)

In [138]:
# 최근 10 게임 포지션 칼럼을 가장 많이 플레이한 포지션 하나로 정리

def mode_is(list):
  return stats.mode(list)[0][0]

def most_position(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_positions_played'
      most_col = f'{t}_' + p + '_most_played'
      wr[col] = wr[col].str.split().apply(mode_is)
      wr.rename(columns={col : most_col}, inplace = True)

most_position(wr)

In [139]:
# 데이터를 100(Blue), 200(Red) 팀으로 나누어서 한 팀의 정보만 담은 데이터셋으로 변환

wr_col100 = wr.columns[['100' in x for x in wr.columns]].tolist()
wr_col200 = wr.columns[['200' in x for x in wr.columns]].tolist()

wr_blue = wr[wr_col100].copy()
wr_red = wr[wr_col200].copy()

wt = 'winning_team'

wr_blue[wt] = wr[wt]
wr_red[wt] = wr[wt]

wr_blue['Team'] = 'Blue'
wr_red['Team'] = 'Red'

# 'Result' column은 0 패배 1 승리

wr_blue['Result'] = 0
wr_blue['Result'][wr_blue[wt] == 100] = 1

wr_red['Result'] = 0
wr_red['Result'][wr_red[wt] == 200] = 1

wr_blue.drop(wt, axis = 1, inplace = True)
wr_red.drop(wt, axis = 1, inplace = True)

wr_blue

,100_TOP_most_played,100_TOP_avg_kda,100_TOP_avg_kp,100_TOP_avg_vision,100_TOP_avg_cs,100_TOP_goldpm,100_TOP_dmgpm,100_TOP_dmg_to_obj_pm,100_TOP_wr,100_TOP_lp,...,100_UTILITY_dmgpm,100_UTILITY_dmg_to_obj_pm,100_UTILITY_wr,100_UTILITY_lp,100_UTILITY_role_total_played,100_UTILITY_role_wr,100_UTILITY_champ_games_played,100_UTILITY_champ_winrate,Team,Result
0,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,563.949624,98.107399,0.3,374.0,1947.0,0.511556,3331,0.507355,Blue,0
1,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,305.332855,103.118608,0.7,820.0,1327.0,0.514695,225,0.542222,Blue,0
2,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,554.237533,147.054926,0.5,601.0,119.0,0.546218,1,0.000000,Blue,0
3,TOP,1.912560,0.368992,16.5,6.744090,424.682704,688.888428,230.514309,0.4,1032.0,...,216.193534,46.067285,0.7,876.0,462.0,0.582251,25,0.600000,Blue,0
4,TOP,3.170368,0.357680,18.0,6.643375,404.560602,473.586402,530.377587,0.5,694.0,...,925.302577,219.550161,0.2,719.0,28.0,0.571429,20,0.600000,Blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,TOP,2.217042,0.408708,19.4,6.754376,427.093702,716.603167,435.780450,0.7,0.0,...,284.884047,128.074746,0.7,1.0,49.0,0.591837,11,0.636364,Blue,1
2084,TOP,5.860595,0.465063,29.1,7.977548,485.142957,852.225925,340.270128,0.6,26.0,...,265.708437,100.961538,0.6,100.0,64.0,0.562500,1,0.000000,Blue,0
2085,MIDDLE,4.081071,0.492928,19.1,5.714827,433.344710,691.186955,142.999621,0.5,0.0,...,193.918540,46.460731,0.3,-62.0,82.0,0.451220,45,0.533333,Blue,0
2086,TOP,3.063929,0.335034,31.8,6.572781,405.116011,575.201431,199.967679,0.6,-232.0,...,255.766753,23.175278,0.5,-32.0,23.0,0.521739,4,0.000000,Blue,0


In [140]:
# colunm 명 변경

def remove_100(x):
  if '100' in x:
    x.remove('100')
  str = '_'.join(x)
  return str

col_united = wr_blue.columns.str.split('_').map(remove_100)

wr_blue.columns = col_united
wr_red.columns = col_united

# 병합

qw = pd.concat([wr_blue, wr_red], ignore_index=True)
qw

,TOP_most_played,TOP_avg_kda,TOP_avg_kp,TOP_avg_vision,TOP_avg_cs,TOP_goldpm,TOP_dmgpm,TOP_dmg_to_obj_pm,TOP_wr,TOP_lp,...,UTILITY_dmgpm,UTILITY_dmg_to_obj_pm,UTILITY_wr,UTILITY_lp,UTILITY_role_total_played,UTILITY_role_wr,UTILITY_champ_games_played,UTILITY_champ_winrate,Team,Result
0,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,563.949624,98.107399,0.3,374.0,1947.0,0.511556,3331,0.507355,Blue,0
1,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,305.332855,103.118608,0.7,820.0,1327.0,0.514695,225,0.542222,Blue,0
2,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,554.237533,147.054926,0.5,601.0,119.0,0.546218,1,0.000000,Blue,0
3,TOP,1.912560,0.368992,16.5,6.744090,424.682704,688.888428,230.514309,0.4,1032.0,...,216.193534,46.067285,0.7,876.0,462.0,0.582251,25,0.600000,Blue,0
4,TOP,3.170368,0.357680,18.0,6.643375,404.560602,473.586402,530.377587,0.5,694.0,...,925.302577,219.550161,0.2,719.0,28.0,0.571429,20,0.600000,Blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,MIDDLE,7.217500,0.485764,19.5,6.350779,432.854070,601.313954,180.336310,1.0,1.0,...,226.222337,89.173495,0.4,-85.0,40.0,0.525000,40,0.550000,Red,0
4172,TOP,3.712962,0.353581,16.0,7.049641,418.459432,612.172544,312.250599,0.7,233.0,...,263.628697,183.463934,0.4,136.0,33.0,0.484848,2,1.000000,Red,1
4173,TOP,4.283333,0.401685,20.9,6.491852,420.580633,517.722420,453.867765,0.5,-131.0,...,446.464098,198.893109,0.7,-77.0,16.0,0.500000,2,0.500000,Red,1
4174,TOP,2.171414,0.397915,21.8,7.569480,433.896037,775.236747,318.335049,0.4,234.0,...,696.190033,138.391942,0.6,-56.0,6.0,0.666667,1,1.000000,Red,1


In [141]:
# 플레이어가 한 번도 해당 포지션을 해본 적이 없는 경우 게임 수, 승률 0 으로 처리
# 처음 하는 챔피언의 승률 0 으로 처리 (champ_games_played = 0 일때 champ_winrate가 결측치)

#na_qw = qw[qw.columns[['champ' in x for x in qw.columns]]]
#na_col = na_qw.isna().sum(axis = 1)[na_qw.isna().sum(axis = 1) != 0].index
#na_qw.iloc[na_col]

qw_col_role = qw.columns[['role' in x for x in qw.columns]].tolist()
qw_col_champ = qw.columns[['champ' in x for x in qw.columns]].tolist()

for col in qw_col_role + qw_col_champ:
  qw[col].fillna(0, inplace = True)

qw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 72 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TOP_most_played             4176 non-null   object 
 1   TOP_avg_kda                 4176 non-null   float64
 2   TOP_avg_kp                  4176 non-null   float64
 3   TOP_avg_vision              4176 non-null   float64
 4   TOP_avg_cs                  4176 non-null   float64
 5   TOP_goldpm                  4176 non-null   float64
 6   TOP_dmgpm                   4176 non-null   float64
 7   TOP_dmg_to_obj_pm           4176 non-null   float64
 8   TOP_wr                      4176 non-null   float64
 9   TOP_lp                      3926 non-null   object 
 10  TOP_role_total_played       4176 non-null   float64
 11  TOP_role_wr                 4176 non-null   float64
 12  TOP_champ_games_played      4176 non-null   int64  
 13  TOP_champ_winrate           4176 

In [142]:
#pr = qw.profile_report()
#pr.to_file('pr_report.html')

In [148]:
# 훈련 데이터, 테스트 데이터 분리

target = 'Result'

X = qw.drop(target, axis = 1)
y = qw[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape

(3340, 71)

In [ ]:
# 시각화

### 3. 모델링 및 모델 해석

In [150]:
# baseline = 0, score = 0.502994

y_train.value_counts(normalize = True)

0    0.502994
1    0.497006
Name: Result, dtype: float64

In [ ]:
pipe = make_pipeline(
    OrdinalEncoder()
    , SimpleImputer(strategy='mean')
    , RandomForestClassifier(n_estimators=500, n_jobs=-1)
)

dists = {
    'randomforestclassifier__max_features': uniform(0, 1)
    , 'randomforestclassifier__min_samples_leaf': randint(1,30)
    }

clf = RandomizedSearchCV(
    estimator = pipe
    , param_distributions = dists
    , scoring='accuracy'
    , cv = 4
    , n_iter = 50
    , n_jobs = -1
    )

clf.fit(X_train, y_train)
print("최적 파라미터 :", clf.best_params_)
print("정확도 :", clf.best_score_)